In [38]:
import pandas as pd

In [39]:
data = pd.read_csv('Language Detection.csv')

In [40]:
data

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English
...,...,...
10332,ನಿಮ್ಮ ತಪ್ಪು ಏನು ಬಂದಿದೆಯೆಂದರೆ ಆ ದಿನದಿಂದ ನಿಮಗೆ ಒ...,Kannada
10333,ನಾರ್ಸಿಸಾ ತಾನು ಮೊದಲಿಗೆ ಹೆಣಗಾಡುತ್ತಿದ್ದ ಮಾರ್ಗಗಳನ್...,Kannada
10334,ಹೇಗೆ ' ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎ...,Kannada
10335,ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ...,Kannada


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vertorizer = TfidfVectorizer()
X = vertorizer.fit_transform(data['Text'])
Y = data['Language']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [42]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [43]:
model.score(X_train, Y_train)

0.9945820433436533

In [44]:
y_pred = model.predict(X_test)

In [45]:
pd.DataFrame(X_test)

,0
0,"(0, 33558)\t0.1894835357091692\n (0, 33095)..."
1,"(0, 20633)\t0.2508358874391975\n (0, 775)\t..."
2,"(0, 7279)\t0.30189822963245944\n (0, 5123)\..."
3,"(0, 30748)\t0.3467426165453571\n (0, 32044)..."
4,"(0, 10802)\t0.277252124680103\n (0, 23942)\..."
...,...
2580,"(0, 38227)\t0.4278531117371759\n (0, 38471)..."
2581,"(0, 29898)\t0.23671476176946157\n (0, 28537..."
2582,"(0, 17454)\t1.0"
2583,"(0, 15962)\t0.35680165578036444\n (0, 20301..."


In [46]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, y_pred)

0.9551257253384913

In [52]:
a = ["Привет, как дела"]
X = vertorizer.transform(a)
prediction = model.predict(X)
print(prediction[0])

Russian
